In [1]:
UNTIE_0 = 0
UNTIE_1 = 1
UNTIE_2 = 2

In [2]:
def print_data(data, num):
    num_numbers = len(data)
    num = min(num, num_numbers)
    print("First {} (from a total of {}) binary numbers:".format(num, num_numbers))
    for code in data[:num]:
        print(code)

In [3]:
def get_processed_input(input_path, data):
    """The data is a square matrix. Each row is a binary number"""
    with open(input_path, "rt") as f:
        input = f.read()
        lines = input.split("\n")
        data += [[int(n) for n in word] for word in lines]
        print_data(data, 10)

In [4]:
def binary_to_decimal(binary):
    dec = 0
    exp = 0
    for digit in binary[::-1]:
        if digit:
            dec += 2**exp
        exp += 1
    return dec

In [5]:
def get_most_common_values(data, untie, num, offset=0):
    """The priority determines the "winner" when there are the same amount of 0s and 1s.
        2 is for when noticing that is needed"""
    if offset + num  > len(data[0]):
        return None
    sums = [dig for dig in data[0][offset:offset+num]]
    for number in data[1:]:
        for i in range(num):
            sums[i] += number[offset + i]
            
    most_common = [0] * num
    half_numbers = len(data) // 2
    pair = len(data) % 2 == 0
    for i in range(num):
        digit = sums[i]
        if digit > half_numbers:
            most_common[i] = 1
        elif pair and digit == half_numbers:
            most_common[i] = untie
    return most_common

In [6]:
def get_gamma_epsilon_rates(data, priority):
    num = len(data[0])
    gamma_rate = get_most_common_values(data, priority, num)
    if not gamma_rate:
        return None, None
    epsilon_rate = [(0 if dig == 1 else 1) for dig in gamma_rate]
    return gamma_rate, epsilon_rate

In [7]:
def part_a(data):
    gamma_rate, epsilon_rate = get_gamma_epsilon_rates(data, UNTIE_0)
    e_rate_dec = binary_to_decimal(epsilon_rate)
    g_rate_dec = binary_to_decimal(gamma_rate)
    return e_rate_dec * g_rate_dec, gamma_rate, epsilon_rate

In [8]:
def part_b(data, verbose=False):
    o2_rating_candidates = [num for num in data]   # Soft copy
    co2_rating_candidates = [num for num in data]
    num_bits = len(data[0])
    o2_found = co2_found = False
    for i in range(num_bits):
        if len(o2_rating_candidates) > 1:
            old_num_candidates = len(o2_rating_candidates)
            most_common_value = get_most_common_values(
                data=o2_rating_candidates,
                untie=UNTIE_2,
                num=1,
                offset=i)[0]  # The returned list has "num" (1) components
            old_candidates = [candidate for candidate in o2_rating_candidates]
            if most_common_value > 1:
                o2_rating_candidates = [candidate for candidate in o2_rating_candidates if candidate[i] == 1]
            else:
                o2_rating_candidates = [candidate for candidate in o2_rating_candidates if candidate[i] == most_common_value]
            # o2_rating_candidates = [candidate for candidate in o2_rating_candidates 
            #                         if candidate[i] == most_common_value or
            #                         (most_common_value == 2 and candidate[i] == 1)]
            if len(o2_rating_candidates) < 1:
                o2_rating_candidates = old_candidates
            if verbose:
                num_candidates = len(o2_rating_candidates) 
                print("O2 -> MCV {}: {} discarded in iteration {}. {} remain".
                     format(most_common_value, old_num_candidates - num_candidates, i, num_candidates))
        elif not o2_found:
            if verbose:
                print("O2 candidate found in iteration " , i-1)
            o2_found = True
            
        if len(co2_rating_candidates) > 1:
            old_num_candidates = len(co2_rating_candidates)
            most_common_value = get_most_common_values(
                data=co2_rating_candidates,
                untie=UNTIE_2,
                num=1,
                offset=i)[0]
            old_candidates = [candidate for candidate in co2_rating_candidates]
            if most_common_value == 2:
                co2_rating_candidates = [candidate for candidate in co2_rating_candidates if candidate[i] == 0]
            else:
                co2_rating_candidates = [candidate for candidate in co2_rating_candidates if candidate[i] != most_common_value]
            # co2_rating_candidates = [candidate for candidate in co2_rating_candidates
            #                         if candidate[i] != most_common_value or
            #                         (most_common_value == 2 and candidate[i] == 0)]
            if len(co2_rating_candidates) < 1:
                co2_rating_candidates = old_candidates
            if verbose:
                num_candidates = len(co2_rating_candidates) 
                print("CO2 -> MCV {}: {} CO2 candidates discarded in iteration {}. {} remain".
                     format(most_common_value, old_num_candidates - num_candidates, i, num_candidates))
        elif not co2_found:
            if verbose:
                print("CO2 candidate found in iteration ", i-1)
            co2_found = True
        if o2_found and co2_found:
            break
    if len(o2_rating_candidates) != 1 or len(co2_rating_candidates) != 1:
        return None, None, None
    if not o2_found and verbose:
            print("O2 candidate found in the last iteration")
    if not co2_found and verbose:
            print("CO2 candidate found in the last iteration")    
    o2_rating_dec = binary_to_decimal(o2_rating_candidates[0])
    co2_rating_dec = binary_to_decimal(co2_rating_candidates[0])
    return o2_rating_dec * co2_rating_dec, o2_rating_candidates[0], co2_rating_candidates[0]

In [9]:
data = []
input_path = "input.txt"
get_processed_input(input_path, data)

First 10 (from a total of 1000) binary numbers:
[1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0]
[1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0]
[1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1]
[0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1]
[0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]
[0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]


In [10]:
sol_a, gamma_rate, epsilon_rate = part_a(data)
print("SOL A: [{}] is the result of multiplying {} ({}) of gamma rate and {} ({}) of epsilon rate".format(
    sol_a, gamma_rate, binary_to_decimal(gamma_rate), epsilon_rate, binary_to_decimal(epsilon_rate)))

SOL A: [749376] is the result of multiplying [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1] (3903) of gamma rate and [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0] (192) of epsilon rate


In [11]:
sol_b, o2_rating, co2_rating = part_b(data, verbose=True)
print("SOL B: [{}] is the result of multiplying {} ({}) of oxygen generator rating and {} ({})\
 of CO2 scrubber rating".format(sol_b, o2_rating, binary_to_decimal(o2_rating), co2_rating,
                                   binary_to_decimal(co2_rating)))

O2 -> MCV 1: 489 discarded in iteration 0. 511 remain
CO2 -> MCV 1: 511 CO2 candidates discarded in iteration 0. 489 remain
O2 -> MCV 1: 248 discarded in iteration 1. 263 remain
CO2 -> MCV 1: 247 CO2 candidates discarded in iteration 1. 242 remain
O2 -> MCV 1: 125 discarded in iteration 2. 138 remain
CO2 -> MCV 0: 124 CO2 candidates discarded in iteration 2. 118 remain
O2 -> MCV 1: 62 discarded in iteration 3. 76 remain
CO2 -> MCV 1: 60 CO2 candidates discarded in iteration 3. 58 remain
O2 -> MCV 0: 34 discarded in iteration 4. 42 remain
CO2 -> MCV 1: 32 CO2 candidates discarded in iteration 4. 26 remain
O2 -> MCV 0: 17 discarded in iteration 5. 25 remain
CO2 -> MCV 0: 14 CO2 candidates discarded in iteration 5. 12 remain
O2 -> MCV 0: 12 discarded in iteration 6. 13 remain
CO2 -> MCV 0: 7 CO2 candidates discarded in iteration 6. 5 remain
O2 -> MCV 1: 6 discarded in iteration 7. 7 remain
CO2 -> MCV 1: 3 CO2 candidates discarded in iteration 7. 2 remain
O2 -> MCV 1: 3 discarded in iterat

## Example test
### A

Gamma rate = 22

Epsilon rate = 9

Consumption = 198

### B

Oxygen generator rating = 23

CO2 scrubber rating = 10

Life support rating = 230

In [12]:
data_test = [[0,0,1,0,0],
[1,1,1,1,0],
[1,0,1,1,0],
[1,0,1,1,1],
[1,0,1,0,1],
[0,1,1,1,1],
[0,0,1,1,1],
[1,1,1,0,0],
[1,0,0,0,0],
[1,1,0,0,1],
[0,0,0,1,0],
[0,1,0,1,0]]

In [13]:
sol_a, gamma_rate, epsilon_rate = part_a(data_test)
print("SOL Test A: [{}] is the result of multiplying {} ({}) of gamma rate and {} ({}) of epsilon rate".format(
    sol_a, gamma_rate, binary_to_decimal(gamma_rate), epsilon_rate, binary_to_decimal(epsilon_rate)))

SOL Test A: [198] is the result of multiplying [1, 0, 1, 1, 0] (22) of gamma rate and [0, 1, 0, 0, 1] (9) of epsilon rate


In [14]:
sol_b, o2_rating, co2_rating = part_b(data_test, verbose=True)
print("SOL Test B: [{}] is the result of multiplying {} ({}) of oxygen generator rating and {} ({})\
 of CO2 scrubber rating".format(sol_b, o2_rating, binary_to_decimal(o2_rating), co2_rating,
                                   binary_to_decimal(co2_rating)))

O2 -> MCV 1: 5 discarded in iteration 0. 7 remain
CO2 -> MCV 1: 7 CO2 candidates discarded in iteration 0. 5 remain
O2 -> MCV 0: 3 discarded in iteration 1. 4 remain
CO2 -> MCV 0: 3 CO2 candidates discarded in iteration 1. 2 remain
O2 -> MCV 1: 1 discarded in iteration 2. 3 remain
CO2 -> MCV 2: 1 CO2 candidates discarded in iteration 2. 1 remain
O2 -> MCV 1: 1 discarded in iteration 3. 2 remain
CO2 candidate found in iteration  2
O2 -> MCV 2: 1 discarded in iteration 4. 1 remain
O2 candidate found in the last iteration
SOL Test B: [230] is the result of multiplying [1, 0, 1, 1, 1] (23) of oxygen generator rating and [0, 1, 0, 1, 0] (10) of CO2 scrubber rating
